In [1]:
# !pip install farm-haystack[all]

In [2]:
# !pip install numpy==1.24

In [3]:
import haystack
from datasets import load_dataset
import numpy as np
print(np.__version__)
print(haystack.__version__)

C:\Users\karol\anaconda3\envs\NLP_haystack\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.24.0
1.26.4


In [4]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever

# document_store = FAISSDocumentStore(similarity="cosine", embedding_dim=768)  # before text loading
document_store = FAISSDocumentStore.load("faiss_document_store2.db")

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="intfloat/multilingual-e5-base", 
    model_format="sentence_transformers",
    top_k=5,
    max_seq_len=512
)

In [5]:
!pip freeze > requirements.txt

C:\Users\karol\anaconda3\envs\NLP_haystack\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
C:\Users\karol\anaconda3\envs\NLP_haystack\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\Users\karol\anaconda3\envs\NLP_haystack\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


In [19]:
ds_qa = load_dataset("clarin-knext/fiqa-pl-qrels")['test'] 
ds_queries = load_dataset("clarin-knext/fiqa-pl","queries")["queries"] 
ds_corpuses = load_dataset("clarin-knext/fiqa-pl", "corpus") 

In [7]:
# documents = [
#     {
#         "content": doc['text'],
#         "id": doc['_id']
#     }
#     for doc in ds_corpuses['corpus']
# ]

# document_store.write_documents(documents)

In [8]:
# document_store.update_embeddings(retriever)  # długo sie robi!!!

In [9]:
# queries = [
#     {
#         "content": query["text"], 
#         "id": query["_id"]
#     }
#     for query in ds_queries
# ]

# query_embeddings = retriever.embed_queries([query["content"] for query in queries])

Code from second lab

In [10]:
from collections import defaultdict, OrderedDict

correct_results = defaultdict(list)

for entry in ds_qa:
    query_id = entry['query-id']
    corpus_id = entry['corpus-id']
    correct_results[query_id].append(corpus_id)

correct_list = [{'query-id': query_id, 'corpuses-id': corpuses} for query_id, corpuses in correct_results.items()]

In [11]:
test_query_ids = set(ds_qa['query-id'])
filtered_queries = OrderedDict(sorted((int(row['_id']), row['text']) for row in ds_queries if int(row['_id']) in test_query_ids))

Searching relevant texts with Faiss and the E5 model:

In [12]:
neural_results = []
k = 5

for query in correct_list:
    text = filtered_queries[query['query-id']]
    top_docs = retriever.retrieve(text, top_k=k)

    for doc in top_docs:
        neural_results.append(
            {
                'query-id': query['query-id'],
                'corpus-id': int(doc.id),
                'analyzer': 0 # for compatibility with code from lab2
        })

Batches: 100%|██████████| 1/1 [00:00<00:00, 11.05it/s]


In [13]:
neural_results = [neural_results] # for comatibility with code from lab2 - I had 4 different analyzers there

In [14]:
neural_results[0][:10]

[{'query-id': 8, 'corpus-id': 65404, 'analyzer': 0},
 {'query-id': 8, 'corpus-id': 309023, 'analyzer': 0},
 {'query-id': 8, 'corpus-id': 1066, 'analyzer': 0},
 {'query-id': 8, 'corpus-id': 316359, 'analyzer': 0},
 {'query-id': 8, 'corpus-id': 590102, 'analyzer': 0},
 {'query-id': 15, 'corpus-id': 325273, 'analyzer': 0},
 {'query-id': 15, 'corpus-id': 472629, 'analyzer': 0},
 {'query-id': 15, 'corpus-id': 224000, 'analyzer': 0},
 {'query-id': 15, 'corpus-id': 99102, 'analyzer': 0},
 {'query-id': 15, 'corpus-id': 233145, 'analyzer': 0}]

In [15]:
def get_DCG(relevance_scores, k):
    return sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevance_scores[:k]))

def get_NDCG(model_results, relevant_documents, k):

    relevance_scores = [1 if doc['corpus-id'] in relevant_documents else 0 for doc in model_results]
    dcg_k = get_DCG(relevance_scores, k)

    num_docs = len(relevant_documents)
    ideal_relevance_scores = [1] * min(num_docs, k) + [0] * (k - min(num_docs, k))
    idcg_k = get_DCG(ideal_relevance_scores, k)

    ndcg_k = dcg_k / idcg_k if idcg_k > 0 else 0
    return ndcg_k

In [16]:
ndcgs = []

for j in range(len(neural_results)):
    results = []
    for i in range(len(correct_list)):
        ndcg = get_NDCG(neural_results[j][i*k:(i+1)*k], correct_list[i]['corpuses-id'], k)
        results.append((correct_list[i]['query-id'], float(ndcg)))

    ndcgs.append(results)

In [17]:
for i in range(len(ndcgs)):
    avg = sum(value for _, value in ndcgs[i]) / len(ndcgs[i])
    print(f"Result after utilizing neural search: {round(avg*100,2)} %")

Result after utilizing neural search: 21.14 %


#### Summary
- [lab 2] Lexical match (lemmatizer, no synonyms): 18.51 %  
- [lab 6] Text clasifier: 21.45 %
- [lab 8] Neural Search: 21.14 %

## Questions

#### 1. Which of the methods: lexical match (e.g. ElasticSearch) or dense representation works better?

Neural search outperformed lexical search by 2.63 percentage points, which is a significant improvement. Therefore, we can conclude that neural search generally works better. However, the text classifier developed in Lab 6 performed slightly better than our new method. However the differences between the methods are small, suggesting that neural search and the classifier are comparable in performance.

#### 2. Which of the method is faster?

Both lexical match and neural search require loading documents into a data structure (index/document-store). While loading the corpus into the document-store took slightly more time, neural search was significantly faster during the actual search process. 

#### 3. Try to determine the other pros and cons of using lexical seaarch and dense document retrieval models.

Retrieval models use vector embeddings to encode information from text, allowing them to capture the semantic meaning of words and phrases. This helps the model 'understand' the context, as similar words or synonyms tend to be closer to each other in the vector space. Leixcal search, however, works using Bag of Words model which ignores word order and it doesn't capture the semantic relationshps between words. 

The pros of using lexical search also includes its interpretability - it is much easier to understand why model matched specific answer.